In [ ]:
import json
import numpy as np
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from llama_cpp import Llama

# ✅ [NEW] LangChain의 프롬프트 템플릿 모듈 가져오기
from langchain_core.prompts import PromptTemplate

# =============================================================================
# [설정] 모델 및 환경
# =============================================================================
MODEL_PATH = "./models/Qwen2.5-Coder-3B-Instruct-Q4_K_M.gguf"
USE_REAL_SLM = False  # 테스트 시 False, 실제 구동 시 True

TARGET_SCHEMA = [
    "user_id", "user_name", "phone_number", "email_address", 
    "signup_date", "last_login", "is_active", "shipping_address"
]

# =============================================================================
# [Prompt Engineering] 프롬프트 템플릿 정의 (여기가 핵심!)
# =============================================================================
# 파이썬 코드 안에 문자열을 숨기지 않고, 전역 변수로 빼서 관리합니다.
# Qwen/Llama3의 포맷(<|im_start|>)을 명시적으로 지켜줍니다.

mapping_template_str = """<|im_start|>system
You are a Data Engineer expert in Schema Mapping.
Select the best Target Column from the Candidates that matches the Source Column.
<|im_end|>
<|im_start|>user
[Input Data]
- Source Column: "{source_col}"
- Sample Data: "{sample_value}"
- Candidates: {candidates}

[Instruction]
Analyze the meaning of the Source Column and the pattern of the Sample Data.
Choose ONE target column. Return ONLY JSON format.

Example Output:
{{
    "selected_column": "target_column_name",
    "reason": "short explanation"
}}
<|im_end|>
<|im_start|>assistant
"""

# LangChain PromptTemplate 객체 생성
MAPPING_PROMPT = PromptTemplate(
    template=mapping_template_str,
    input_variables=["source_col", "sample_value", "candidates"]
)

# =============================================================================
# [초기화] 모델 로딩
# =============================================================================
print("⏳ Loading Models...")
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
target_embeddings = embed_model.encode(TARGET_SCHEMA)

llm = None
if USE_REAL_SLM:
    llm = Llama(model_path=MODEL_PATH, n_ctx=2048, verbose=False)

# =============================================================================
# [Node Logic]
# =============================================================================
class MappingState(TypedDict):
    source_col: str
    sample_value: str
    candidates: List[str]
    final_mapping: str
    reasoning: str

def retriever_node(state: MappingState):
    # (이전과 동일: 임베딩 검색)
    source_vec = embed_model.encode([state["source_col"]])
    similarities = cosine_similarity(source_vec, target_embeddings)[0]
    top_k_indices = np.argsort(similarities)[-3:][::-1]
    candidates = [TARGET_SCHEMA[i] for i in top_k_indices]
    
    print(f"🔍 [Retriever] 후보군 추출: {candidates}")
    return {"candidates": candidates}

def slm_reasoning_node(state: MappingState):
    """
    PromptTemplate을 사용하여 코드가 훨씬 깔끔해진 노드입니다.
    """
    print(f"🧠 [SLM] 추론 시작...")

    # ✅ [NEW] 템플릿에 데이터 주입 (Validation 기능 포함)
    # 변수가 누락되면 여기서 바로 에러를 내주므로 디버깅이 편함
    final_prompt = MAPPING_PROMPT.format(
        source_col=state["source_col"],
        sample_value=state["sample_value"],
        candidates=state["candidates"]
    )
    
    # 실제 추론
    if USE_REAL_SLM and llm:
        output = llm(final_prompt, max_tokens=200, stop=["<|im_end|>"], echo=False)
        response_text = output['choices'][0]['text'].strip()
    else:
        # 시뮬레이션
        response_text = '{"selected_column": "phone_number", "reason": "Simulated Result"}'

    # 파싱 로직 (Robustness 유지)
    try:
        cleaned_text = response_text.replace("```json", "").replace("```", "").strip()
        result_json = json.loads(cleaned_text)
        final_col = result_json.get("selected_column", "Unknown")
        reason = result_json.get("reason", "No reason")
    except json.JSONDecodeError:
        final_col = "Error"
        reason = response_text

    return {"final_mapping": final_col, "reasoning": reason}

# =============================================================================
# [Graph]
# =============================================================================
workflow = StateGraph(MappingState)
workflow.add_node("retriever", retriever_node)
workflow.add_node("reasoner", slm_reasoning_node)
workflow.set_entry_point("retriever")
workflow.add_edge("retriever", "reasoner")
workflow.add_edge("reasoner", END)
app = workflow.compile()

# =============================================================================
# [Test]
# =============================================================================
if __name__ == "__main__":
    result = app.invoke({
        "source_col": "m_hp", 
        "sample_value": "010-1234-5678"
    })
    print(f"\n✅ Result: {result['final_mapping']} ({result['reasoning']})")